In [ ]:
from model import Classifier
from dataLoader import train, test

In [1]:
from facialLandmarkDetector import FacialLandmarkDetection
p = FacialLandmarkDetection("FaceShape Dataset/training_set/Oval/oval (96).jpg")
print(p.load().shape)

2024-04-06 20:14:23.961907: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-06 20:14:24.074313: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/skycoder/.local/lib/python3.10/site-packages/cv2/../../lib64:
2024-04-06 20:14:24.074336: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-04-06 20:14:24.600716: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dle

(478, 2)


I0000 00:00:1712427265.714466  594619 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1712427265.716579  594696 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: RENOIR (renoir, LLVM 15.0.7, DRM 3.54, 6.5.0-26-generic)
W0000 00:00:1712427265.717037  594619 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
